In [1]:
import matplotlib
%matplotlib widget
from matplotlib import pyplot as plt
import psycopg2 as pg2
import psycopg2.extras
import psycopg2.extensions
import pickle
import pandas as pd
import numpy as np
import pint_pandas
import tqdm
import os
import pint
u = pint.UnitRegistry()

# representation
* load all example data sets into the data structure at once
* measure memory use
* export the data sets to CSV
* reimport from CSV
* export the data sets to some kind of native storage format
* reimport from the native storage format


## read in the data

In [2]:
conn = pg2.connect("dbname=ev", cursor_factory=pg2.extras.NamedTupleCursor)

In [3]:
cur = conn.cursor()

In [92]:
def open_dequantify(fpath, deq_col):
    with open(fpath, 'rb') as fd:
        df = pickle.load(fd)
    df['Unit'] = str(df[deq_col].pint.units)
    df[deq_col] = df[deq_col].pint.magnitude
    return df

In [88]:
fao_emi_total_pd = open_dequantify('mem/fao_emi_total_wo.pck', 'Emissions')

In [59]:
cur.execute("""create table fao_emi_total (
Area text,
Date date,
Element text,
Item text,
Emissions double precision,
Unit text,
primary key (Area, Date, Element, Item)
)""")

In [60]:
cur.executemany("""insert into fao_emi_total (Area, Date, Element, Item, Emissions, Unit) values (%s, %s, %s, %s, %s, %s)""",
                (fao_emi_total_pd.itertuples(index=False, name=None)))

In [89]:
lak_emi_energy_pd = open_dequantify('mem/lak_emi_energy_wo.pck', 'Emissions')

In [73]:
cur.execute("""create table lak_emi_energy (
Area text,
Category text,
Date date,
Emissions double precision,
Unit text,
primary key (Area, Category, Date)
)""")

In [75]:
cur.executemany("""insert into lak_emi_energy (Area, Category, Date, Emissions, Unit) values (%s, %s, %s, %s, %s)""",
                (lak_emi_energy_pd.itertuples(index=False, name=None)))

In [76]:
conn.commit()

In [90]:
cmip_conc_co2_pd = open_dequantify('mem/cmip_conc_co2_wo.pck', 'CO2 concentration')

In [50]:
def adapt_period_cmip(period):
    if period.year < 1:
        year = 1 - period.year
        suffix = 'BC'
    else:
        year = period.year
        suffix = ''
    return pg2.extensions.AsIs(f"'{year:04d}-{period.month:02d}-01{suffix}'")

pg2.extensions.register_adapter(pd._libs.tslibs.period.Period, adapt_period_cmip)

In [51]:
def cast_date(value, cur):
    if value is None:
        return None
    
    if value.endswith('BC'):
        bc = True
        value = value.split()[0]
    else:
        bc = False
    
    y, m, d = value.split('-')
    y = int(y)
    if bc:
        y = 1-y  # np.datetime64 counts everything AD and has the year 0 AD
    return np.datetime64(f'{y:04d}-{m}-{d}')

extended_date = pg2.extensions.new_type((1082, ), 'date', cast_date)
pg2.extensions.register_type(extended_date)

In [126]:
cur.execute("""create table cmip_conc_co2 (
Area text,
Date date,
"CO2 concentration" double precision,
Unit text,
primary key (Area, Date)
)""")

In [128]:
cur.executemany("""insert into cmip_conc_co2 (Area, Date, "CO2 concentration", Unit) values (%s, %s, %s, %s)""",
                (cmip_conc_co2_pd.itertuples(index=False, name=None)))

In [129]:
conn.commit()

In [188]:
prm_emi_pd = open_dequantify('mem/prm_emi_wo.pck', 'Emissions')

In [189]:
prm_emi_pd

,Area,Category,Date,Entity,Scenario,Emissions,Unit
0,ABW,IPC1,1850-01-01,CH4,HISTCR,0.000387,gigagram
1,ABW,IPC1,1850-01-01,CH4,HISTTP,0.000387,gigagram
2,ABW,IPC1,1850-01-01,CO2,HISTCR,40.300000,gigagram
3,ABW,IPC1,1850-01-01,CO2,HISTTP,40.300000,gigagram
4,ABW,IPC1,1850-01-01,KYOTOGHG (CO2eq),HISTCR,40.300000,gigagram
...,...,...,...,...,...,...,...
4990919,ZWE,IPCMAGELV,2017-01-01,KYOTOGHG (CO2eq),HISTTP,6630.000000,gigagram
4990920,ZWE,IPCMAGELV,2017-01-01,KYOTOGHGAR4 (CO2eq),HISTCR,6040.000000,gigagram
4990921,ZWE,IPCMAGELV,2017-01-01,KYOTOGHGAR4 (CO2eq),HISTTP,6640.000000,gigagram
4990922,ZWE,IPCMAGELV,2017-01-01,N2O,HISTCR,15.600000,gigagram


In [190]:
cur.execute("""create table prm_emi (
Area text,
Category text,
Date date,
Entity text,
Scenario text,
Emissions double precision,
Unit text,
primary key (Area, Category, Date, Entity, Scenario)
)""")

In [192]:
cur.executemany("""insert into prm_emi (Area, Category, Date, Entity, Scenario, Emissions, Unit) values (%s, %s, %s, %s, %s, %s, %s)""",
                (prm_emi_pd.itertuples(index=False, name=None)))

In [194]:
conn.commit()

In [220]:
with open(f'mem/primap_tables_wo.pck', 'rb') as fd:
    primap_tables_pd = pickle.load(fd)
    

In [221]:
cur.execute("""create table primap_metadata (
primap_metadata_id serial unique,
Category text,
Class text,
Description text,
Note text,
Scenario text,
Source text,
Type text,
Entity text,
Unit text,
primary key (Category, Class, Scenario, Source, Type, Entity)
)""")

In [222]:
cur.execute("""create table primap_data (
primap_metadata_id integer references primap_metadata (primap_metadata_id),
Area text,
Date date,
Value double precision,
primary key (primap_metadata_id, Area, Date)
)""")

In [223]:
for key in tqdm.tqdm_notebook(primap_tables_pd):
    meta, df = primap_tables_pd[key]
    
    entity = df.columns[-1]
    unit = str(df[entity].pint.units)
    df[entity] = df[entity].pint.magnitude
    
    cur.execute("""insert into primap_metadata (Category, Class, Description, Note, Scenario, Source, Type, Entity, Unit)
                   values (%s, %s, %s, %s, %s, %s, %s, %s, %s)
                   returning primap_metadata_id""",
                (meta['category'], meta['class'], meta['descr'], meta['note'], meta['scenario'], meta['source'], meta['type'], entity, unit))
    metadata_id = cur.fetchone()[0]
    
    cur.executemany(f"""insert into primap_data (primap_metadata_id, Area, Date, Value)
                        values ({metadata_id}, %s, %s, %s)""",
                    df.itertuples(index=False, name=None))

In [224]:
conn.commit()

## Measure memory use

In [225]:
!systemctl status postgresql@12-main.service

● postgresql@12-main.service - PostgreSQL Cluster 12-main
     Loaded: loaded (]8;;file://lt874/lib/systemd/system/postgresql@.service/lib/systemd/system/postgresql@.service]8;;; enabled-runtime; vendor preset: enabled)
     Active: active (running) since Mon 2020-09-21 15:34:07 CEST; 20h ago
   Main PID: 23721 (postgres)
      Tasks: 16 (limit: 18915)
     Memory: 2.3G
     CGroup: /system.slice/system-postgresql.slice/postgresql@12-main.service
             ├─ 23721 /usr/lib/postgresql/12/bin/postgres -D /var/lib/postgresq…
             ├─ 23723 postgres: 12/main: checkpointer
             ├─ 23724 postgres: 12/main: background writer
             ├─ 23725 postgres: 12/main: walwriter
             ├─ 23726 postgres: 12/main: autovacuum launcher
             ├─ 23727 postgres: 12/main: stats collector
             ├─ 23728 postgres: 12/main: logical replication launcher
             ├─ 27206 postgres: 12/main: pflueger ev [local] idle
             ├─ 34534 postgres: 12/main: pflue

In [233]:
sizes = pd.read_sql_query("""SELECT relname, relpages * 8 as relsize_kib
FROM pg_class
where relnamespace = 2200
ORDER BY relpages DESC;""",
                 con="postgres:///ev")
sizes

,relname,relsize_kib
0,primap_data,681688
1,prm_emi,438696
2,primap_data_pkey,412864
3,prm_emi_pkey,271448
4,fao_emi_total,50456
5,fao_emi_total_pkey,39912
6,cmip_conc_co2,4464
7,cmip_conc_co2_pkey,1816
8,primap_metadata,1432
9,primap_metadata_pkey,384


In [235]:
sizes['relsize_kib'].sum() / 1024  # MiB

1859.0625

In [236]:
!ls -lah mem/

total 674M
drwxrwxr-x  2 pflueger pflueger 4,0K Aug 27 10:55 .
drwxrwxr-x 10 pflueger pflueger 4,0K Sep 22 12:14 ..
-rw-rw-r--  1 pflueger pflueger 1,2M Aug 27 18:49 cmip_conc_co2.pck
-rw-rw-r--  1 pflueger pflueger 1,2M Aug 27 18:53 cmip_conc_co2_wo.pck
-rw-rw-r--  1 pflueger pflueger 8,8M Aug 27 18:49 fao_emi_total.pck
-rw-rw-r--  1 pflueger pflueger 9,7M Aug 27 18:53 fao_emi_total_wo.pck
-rw-rw-r--  1 pflueger pflueger  40K Aug 27 18:49 lak_emi_energy.pck
-rw-rw-r--  1 pflueger pflueger  50K Aug 27 18:53 lak_emi_energy_wo.pck
-rw-rw-r--  1 pflueger pflueger 205M Aug 27 18:49 primap_tables.pck
-rw-rw-r--  1 pflueger pflueger 244M Aug 27 18:53 primap_tables_wo.pck
-rw-rw-r--  1 pflueger pflueger 105M Aug 27 18:49 prm_emi.pck
-rw-rw-r--  1 pflueger pflueger 100M Aug 27 18:53 prm_emi_wo.pck


## I/O
* export the data sets to CSV
* reimport from CSV
* export the data sets to some kind of native storage format
* reimport from the native storage format

In [247]:
for table in ('prm_emi', 'fao_emi_total', 'cmip_conc_co2', 'lak_emi_energy'):
    with open(f'psql/{table}.csv', 'wb') as fd:
        cur.copy_expert(f"""copy {table}
        to stdout
        ( format csv, header true )
        """, fd)

In [248]:
!ls -la psql/

total 300504
drwxrwxr-x  2 pflueger pflueger      4096 Sep 22 18:00 .
drwxrwxr-x 11 pflueger pflueger      4096 Sep 22 18:00 ..
-rw-rw-r--  1 pflueger pflueger   3697625 Sep 22 18:00 cmip_conc_co2.csv
-rw-rw-r--  1 pflueger pflueger  34561578 Sep 22 18:00 fao_emi_total.csv
-rw-rw-r--  1 pflueger pflueger    188021 Sep 22 18:00 lak_emi_energy.csv
-rw-rw-r--  1 pflueger pflueger 269255848 Sep 22 18:00 prm_emi.csv


In [251]:
cur.execute("""select primap_metadata_id from primap_metadata""")
for pmid, in tqdm.tqdm_notebook(cur.fetchall()):
    with open(f'psql/primap/{pmid}.csv', 'wb') as fd:
        cur.copy_expert(f"""copy (select * from primap_data where primap_metadata_id = {pmid})
        to stdout
        ( format csv, header true )
        """, fd)
        
with open('psql/primap_metadata.csv', 'wb') as fd:
    cur.copy_expert("""copy primap_metadata to stdout ( format csv, header true)""", fd)

In [275]:
# reimport
# copy from does not infer data types or create tables
# that is annoying, so have to do some stuff to create tables
data_types = {'date': 'date', 'emissions': 'double precision', 'CO2 concentration': 'double precision'}

for table in ('prm_emi', 'fao_emi_total', 'cmip_conc_co2', 'lak_emi_energy'):
    with open(f'psql/{table}.csv', 'rb') as fd:
        fline = fd.readline().decode()
        headers = fline.strip().split(',')
        header_spec = ', '.join((f'"{header}" {data_types.get(header, "text")}' for header in headers))
        primary_key = ', '.join((header for header in headers if header not in ['emissions', 'CO2 concentration', 'unit']))
        
        sql = f"""create table {table}_r ({header_spec}, primary key ({primary_key}))"""
        cur.execute(sql)
        
        cur.copy_expert(f"""copy {table}_r from stdout ( format csv, header false )""", fd)

In [277]:
cur.execute("""create table primap_metadata_r (
primap_metadata_id serial unique,
Category text,
Class text,
Description text,
Note text,
Scenario text,
Source text,
Type text,
Entity text,
Unit text,
primary key (Category, Class, Scenario, Source, Type, Entity)
)""")

with open('psql/primap_metadata.csv', 'rb') as fd:
    cur.copy_expert("""copy primap_metadata_r from stdout ( format csv, header true)""", fd)
    

In [281]:
cur.execute("""create table primap_data_r (
primap_metadata_id integer references primap_metadata (primap_metadata_id),
Area text,
Date date,
Value double precision,
primary key (primap_metadata_id, Area, Date)
)""")

for fpath in tqdm.tqdm_notebook(os.listdir('psql/primap/')):
    with open(f'psql/primap/{fpath}', 'rb') as fd:
        cur.copy_expert(f"""copy primap_data_r
        from stdout
        ( format csv, header true )
        """, fd)
        

In [282]:
conn.commit()

In [8]:
# native format: pg_dump
!mkdir psql/dump/
!pg_dump --format=directory -j4 -t prm_emi -t fao_emi_total -t cmip_conc_co2 -t lak_emi_energy -t primap_metadata -t primap_data -v ev -f psql/dump/

mkdir: cannot create directory ‘psql/dump/’: File exists
pg_dump: last built-in OID is 16383
pg_dump: reading extensions
pg_dump: identifying extension members
pg_dump: reading schemas
pg_dump: reading user-defined tables
pg_dump: reading user-defined functions
pg_dump: reading user-defined types
pg_dump: reading procedural languages
pg_dump: reading user-defined aggregate functions
pg_dump: reading user-defined operators
pg_dump: reading user-defined access methods
pg_dump: reading user-defined operator classes
pg_dump: reading user-defined operator families
pg_dump: reading user-defined text search parsers
pg_dump: reading user-defined text search templates
pg_dump: reading user-defined text search dictionaries
pg_dump: reading user-defined text search configurations
pg_dump: reading user-defined foreign-data wrappers
pg_dump: reading user-defined foreign servers
pg_dump: reading default privileges
pg_dump: reading user-defined collations
pg_dump: reading user-defined conversions
pg_

In [9]:
!du -sh psql/dump/*

3,5M	psql/dump/3020.dat.gz
36K	psql/dump/3021.dat.gz
776K	psql/dump/3022.dat.gz
25M	psql/dump/3023.dat.gz
96K	psql/dump/3025.dat.gz
124M	psql/dump/3026.dat.gz
12K	psql/dump/toc.dat


In [10]:
!pg_restore --format=directory -j4 --clean -d ev -v psql/dump

pg_restore: connecting to database for restore
pg_restore: dropping FK CONSTRAINT primap_data primap_data_primap_metadata_id_fkey
pg_restore: dropping CONSTRAINT prm_emi prm_emi_pkey
pg_restore: dropping CONSTRAINT primap_metadata primap_metadata_primap_metadata_id_key
pg_restore: while PROCESSING TOC:
pg_restore: from TOC entry 2890; 2606 16533 CONSTRAINT primap_metadata primap_metadata_primap_metadata_id_key pflueger
pg_restore: error: could not execute query: ERROR:  cannot drop constraint primap_metadata_primap_metadata_id_key on table public.primap_metadata because other objects depend on it
DETAIL:  constraint primap_data_r_primap_metadata_id_fkey on table public.primap_data_r depends on index public.primap_metadata_primap_metadata_id_key
HINT:  Use DROP ... CASCADE to drop the dependent objects too.
Command was: ALTER TABLE ONLY public.primap_metadata DROP CONSTRAINT primap_metadata_primap_metadata_id_key;
pg_restore: dropping CONSTRAINT primap_metadata primap_metadata_pkey
pg_r

# interactive analysis and plotting
* add / subtract / divide / multiply compatible, overlapping data sets
* timeit
* plot a historical time series, and something akin to gapminder (countries on a gdp vs emissions plot)
* select datasets which contain data on the N2O emissions of Finland
* in a selected dataset, select years in which the N2O emissions of Finland were above the 1950-2000 average
* resample non-yearly dataset to yearly dataset
* convert a dataset to tidy format

In [25]:
# extract two compatible, partly overlapping data sets from primap-hist

In [7]:
cur.execute('drop view if exists a')
cur.execute("""
create view a as 
    select prm_emi.*, emissions * unit::unit as emissions_q from prm_emi
    where Category = 'IPC1' and
          date < '1991-01-01'::date and
          date >= '1900-01-01'::date and
          Entity = 'CO2' and
          Scenario = 'HISTCR' """)

In [8]:
cur.execute('drop view if exists ar')
cur.execute("""
create view ar as 
    select prm_emi_r.*, emissions * unit::unit as emissions_q from prm_emi_r
    where Category = 'IPC1' and date < '1991-01-01'::date and date >= '1900-01-01'::date and Entity = 'CO2' and Scenario = 'HISTCR' """)

In [9]:
cur.execute('drop view if exists b')
cur.execute("""create view b as 
                  select prm_emi.*, emissions * unit::unit as emissions_q from prm_emi
                  where area in ('DEU', 'FIN', 'ZWE') """)

In [10]:
conn.commit()

In [5]:
def pdq(query):
    return pd.read_sql_query(query, 'postgres:///ev')

In [12]:
pdq("""select a.emissions + b.emissions as results
       from a join b using (area, category, date, entity, scenario)""")
# note the timing includes conversion into a pandas dataframe, the actual query is on the order of 3 ms

,results
0,640000.0
1,638000.0
2,626000.0
3,674000.0
4,698000.0
...,...
268,22800.0
269,26000.0
270,27200.0
271,27400.0


In [13]:
pdq("""select ar.emissions + b.emissions as results
       from ar join b using (area, category, date, entity, scenario)""")

,results
0,640000.0
1,638000.0
2,626000.0
3,674000.0
4,698000.0
...,...
268,22800.0
269,26000.0
270,27200.0
271,27400.0


In [14]:
pdq("""select a.emissions - b.emissions as results
       from a join b using (area, category, date, entity, scenario)""")

,results
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0
...,...
268,0.0
269,0.0
270,0.0
271,0.0


In [15]:
pdq("""select a.emissions * b.emissions as results
       from a join b using (area, category, date, entity, scenario)""")

,results
0,1.024000e+11
1,1.017610e+11
2,9.796900e+10
3,1.135690e+11
4,1.218010e+11
...,...
268,1.299600e+08
269,1.690000e+08
270,1.849600e+08
271,1.876900e+08


In [16]:
pdq("""select a.emissions / b.emissions as results
       from a join b using (area, category, date, entity, scenario)""")

,results
0,1.0
1,1.0
2,1.0
3,1.0
4,1.0
...,...
268,1.0
269,1.0
270,1.0
271,1.0


In [27]:
# with units
pdq("""select a.emissions_q + b.emissions_q as results
       from a join b using (area, category, date, entity, scenario)""")

,results
0,640 Tg
1,638 Tg
2,626 Tg
3,674 Tg
4,698 Tg
...,...
268,22.8 Tg
269,26 Tg
270,27.2 Tg
271,27.4 Tg


In [34]:
## needed for straight-up cur.fetchall to work on quantities with units
def cast_unit(value, cur):
    if value is None:
        return None
    
    return u(value)

cur.execute("SELECT '12 m'::unit")
oid = cur.description[0][1]

unit = pg2.extensions.new_type((oid, ), 'unit', cast_unit)
pg2.extensions.register_type(unit)

## plotting
* plot a historical time series, and something akin to gapminder (countries on a gdp vs emissions plot)
* resample non-yearly dataset to yearly dataset

In [ ]:
plt.figure()
sel = prm_emi.xs(('DEU', 'IPCM0EL', 'KYOTOGHG (CO2eq)', 'HISTCR'), level=('Area', 'Category', 'Entity', 'Scenario'))
#sel = prm_emi.loc['DEU', 'IPCM0EL', :, 'KYOTOGHG (CO2eq)', 'HISTCR']
sel.pint.magnitude.plot()

In [10]:
pdq("""select prm_emi.emissions from prm_emi where
       Area = 'DEU' and Category = 'IPCM0EL' and Entity = 'KYOTOGHG (CO2eq)' and Scenario = 'HISTCR'""")\
.plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [15]:
pdq("select * from primap_metadata where entity = 'POP' and source = 'UN2019P' and scenario = 'HISTORY'")

,primap_metadata_id,category,class,description,note,scenario,source,type,entity,unit
0,3933,DEMOGR,TOTAL,Medium-fertility variant,Read in by JG,HISTORY,UN2019P,NET,POP,dimensionless


In [33]:
cur.execute("drop view if exists emi_pop_2000")
cur.execute("drop view if exists pop_2000")
cur.execute("create view pop_2000 as select * from primap_data where primap_metadata_id = 3933 and date = '2000-01-01'")
cur.execute("drop view if exists emi_2000")
cur.execute("""
create view emi_2000 as 
    select * from prm_emi 
    where category = 'IPCM0EL'
    and entity = 'KYOTOGHG (CO2eq)'
    and scenario = 'HISTCR'
    and date = '2000-01-01'""")
cur.execute("""
create view emi_pop_2000 as
    select pop_2000.value as pop, emi_2000.emissions as emi, area
    from pop_2000 join emi_2000 using (area, date)""")
conn.commit()

In [35]:
df = pdq("select * from emi_pop_2000")
df.plot.scatter(x='pop', y='emi', label='area')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Selecting datasets and data
* select datasets which contain data on the N2O emissions of Finland
* in a selected dataset, select years in which the N2O emissions of Finland were above the 1950-2000 average

In [37]:
pdq("""
select distinct primap_metadata_id, description, scenario, source
from primap_metadata join primap_data using (primap_metadata_id)
where primap_metadata.entity = 'N2O'
  and primap_data.area = 'FIN'
  and primap_metadata.category = 'IPC0'
order by primap_metadata_id
""")

,primap_metadata_id,description,scenario,source
0,2996,Result of N2O_IPC5_TOTAL_NET_HISTCR_PRIMAPHIST...,HISTCR,PRIMAPHIST20
1,3000,Result of N2O_IPC5+IPC4+IPC3+IPC2_TOTAL_NET_HI...,HISTORY,CRF2019
2,3001,Result of N2O_IPC5+IPC4+IPC3+IPC2_TOTAL_NET_HI...,HISTORY,CRF2020
3,3002,Result of N2O_CAT7+CAT6+CAT5+CAT4+CAT3+CAT2_TO...,HISTORY,EDGAR42COMPI
4,3003,Result of N2O_CAT7+CAT6+CAT4+CAT3+CAT2_TOTAL_N...,HISTORY,EDGAR432I
5,3004,Result of N2O_IPC5+IPC4+IPC3+IPC2_TOTAL_NET_HI...,HISTORY,EDGAR50I
6,3007,Result of N2O_IPC5_TOTAL_NET_HISTTP_PRIMAPHIST...,HISTTP,PRIMAPHIST20


In [46]:
cur.execute("drop view if exists fin_emi")
cur.execute("create view fin_emi as select date, value from primap_data where primap_metadata_id = 3007 and area = 'FIN'")
conn.commit()
pdq("""
select * from fin_emi
where value > (select avg(value) from fin_emi where date >= '1950-01-01' and date <= '2000-01-01')
""")

,date,value
0,1968-01-01,23.505047
1,1969-01-01,23.722403
2,1970-01-01,23.886290
3,1971-01-01,24.266094
4,1972-01-01,25.070313
5,1973-01-01,26.521749
6,1974-01-01,27.165970
7,1975-01-01,26.874863
8,1976-01-01,24.481267
9,1977-01-01,23.781692


## Resampling
* resample non-yearly dataset to yearly dataset

In [71]:
pdq("""
select extract(year from date) as year, area, avg("CO2 concentration")
from cmip_conc_co2 
group by extract(year from date), area
order by year, area
""")

,year,area,avg
0,-1.0,global,0.000277
1,-1.0,nh,0.000277
2,-1.0,sh,0.000277
3,1.0,global,0.000277
4,1.0,nh,0.000277
...,...,...,...
6040,2013.0,nh,0.000398
6041,2013.0,sh,0.000394
6042,2014.0,global,0.000398
6043,2014.0,nh,0.000400


# Interpolate
* delete some data points from a data set and fill them in using linear interpolation

In [80]:
#cur.execute("drop view if exists fin_emi")
cur.execute("drop table if exists fin_emi")
cur.execute("create table fin_emi as select date, value from primap_data where primap_metadata_id = 3007 and area = 'FIN'")
conn.commit()

In [88]:
cur.execute("update fin_emi set value = NULL where date >= '1990-01-01' and date < '1995-01-01'")

In [90]:
# has to be done via pandas
pdq("select * from fin_emi").interpolate()

,date,value
0,1850-01-01,1.149008
1,1851-01-01,1.192368
2,1852-01-01,1.235104
3,1853-01-01,1.276830
4,1854-01-01,1.317161
...,...,...
162,2012-01-01,25.191643
163,2013-01-01,26.698377
164,2014-01-01,26.450066
165,2015-01-01,27.569375


# alternative: sqlite

In [3]:
!mkdir -p sqlite/

import sqlite3
conn = sqlite3.connect(#':memory:', # could also be somewhere instead of in memory
                       'sqlite/data.sqlite3',
                       detect_types=sqlite3.PARSE_DECLTYPES | sqlite3.PARSE_COLNAMES)
cur = conn.cursor()
import datetime

In [176]:
fao_emi_total_pd = open_dequantify('mem/fao_emi_total_wo.pck', 'Emissions')
fao_emi_total_pd['Date'] = [x.date() for x in pd.to_datetime(fao_emi_total_pd['Date'])]

cur.execute("""create table fao_emi_total (
Area text,
Date date,
Element text,
Item text,
Emissions double precision,
Unit text,
primary key (Area, Date, Element, Item)
)""")
cur.executemany("""insert into fao_emi_total (Area, Date, Element, Item, Emissions, Unit) values (?, ?, ?, ?, ?, ?)""",
                (fao_emi_total_pd.itertuples(index=False, name=None)))

In [177]:
lak_emi_energy_pd = open_dequantify('mem/lak_emi_energy_wo.pck', 'Emissions')
lak_emi_energy_pd['Date'] = [x.date() for x in pd.to_datetime(lak_emi_energy_pd['Date'])]

cur.execute("""create table lak_emi_energy (
Area text,
Category text,
Date date,
Emissions double precision,
Unit text,
primary key (Area, Category, Date)
)""")
cur.executemany("""insert into lak_emi_energy (Area, Category, Date, Emissions, Unit) values (?, ?, ?, ?, ?)""",
                (lak_emi_energy_pd.itertuples(index=False, name=None)))

In [178]:
cmip_conc_co2_pd = open_dequantify('mem/cmip_conc_co2_wo.pck', 'CO2 concentration')
cmip_conc_co2_pd['year'] = [x.year for x in cmip_conc_co2_pd['Date']]
cmip_conc_co2_pd['month'] = [x.month for x in cmip_conc_co2_pd['Date']]

In [179]:
del cmip_conc_co2_pd['Date']

In [180]:
cmip_conc_co2_pd

,Area,CO2 concentration,Unit,year,month
0,global,0.000278,dimensionless,0,1
1,global,0.000279,dimensionless,0,2
2,global,0.000279,dimensionless,0,3
3,global,0.000279,dimensionless,0,4
4,global,0.000279,dimensionless,0,5
...,...,...,...,...,...
72535,sh,0.000396,dimensionless,2014,8
72536,sh,0.000396,dimensionless,2014,9
72537,sh,0.000396,dimensionless,2014,10
72538,sh,0.000396,dimensionless,2014,11


In [181]:
cur.execute("""create table cmip_conc_co2 (
Area text,
"CO2 concentration" double precision,
Unit text,
year int,
month int,
primary key (Area, year, month)
)""")
cur.executemany("""insert into cmip_conc_co2 (Area, "CO2 concentration", Unit, year, month) values (?, ?, ?, ?, ?)""",
                (cmip_conc_co2_pd.itertuples(index=False, name=None)))

In [182]:
prm_emi_pd = open_dequantify('mem/prm_emi_wo.pck', 'Emissions')
prm_emi_pd['Date'] = [x.date() for x in pd.to_datetime(prm_emi_pd['Date'])]

cur.execute("""create table prm_emi (
Area text,
Category text,
Date date,
Entity text,
Scenario text,
Emissions double precision,
Unit text,
primary key (Area, Category, Date, Entity, Scenario)
)""")
cur.executemany("""insert into prm_emi (Area, Category, Date, Entity, Scenario, Emissions, Unit) values (?, ?, ?, ?, ?, ?, ?)""",
                (prm_emi_pd.itertuples(index=False, name=None)))

In [183]:
with open(f'mem/primap_tables_wo.pck', 'rb') as fd:
    primap_tables_pd = pickle.load(fd)
    
cur.execute("""create table primap_metadata (
primap_metadata_id integer primary key,
Category text,
Class text,
Description text,
Note text,
Scenario text,
Source text,
Type text,
Entity text,
Unit text,
unique (Category, Class, Scenario, Source, Type, Entity)
)""")

cur.execute("""create table primap_data (
primap_metadata_id integer references primap_metadata (primap_metadata_id),
Area text,
Date date,
Value double precision,
primary key (primap_metadata_id, Area, Date)
)""")

for key in tqdm.tqdm_notebook(primap_tables_pd):
    meta, df = primap_tables_pd[key]
    
    entity = df.columns[-1]
    unit = str(df[entity].pint.units)
    df[entity] = df[entity].pint.magnitude
    try:
        df['Date'] = [x.date() for x in pd.to_datetime(df['Date'])]
    except:
        continue
    
    cur.execute("""insert into primap_metadata (Category, Class, Description, Note, Scenario, Source, Type, Entity, Unit)
                   values (?, ?, ?, ?, ?, ?, ?, ?, ?)""",
                (meta['category'], meta['class'], meta['descr'], meta['note'], meta['scenario'], meta['source'], meta['type'], entity, unit))
    cur.execute("""select last_insert_rowid()""")
    metadata_id = cur.fetchone()[0]
    
    
    cur.executemany(f"""insert into primap_data (primap_metadata_id, Area, Date, Value)
                        values ({metadata_id}, ?, ?, ?)""",
                    df.itertuples(index=False, name=None))
conn.commit()

In [189]:
!ls -lah sqlite/
# this is the native i/o format

total 1,4G
drwxrwxr-x  2 pflueger pflueger 4,0K Sep 23 19:43 .
drwxrwxr-x 12 pflueger pflueger 4,0K Sep 23 19:43 ..
-rw-r--r--  1 pflueger pflueger 1,4G Sep 23 19:43 data.sqlite3


In [190]:
# do aligned arithmetic to get a feeling for performance and features

In [13]:
cur.execute('drop view if exists a')
cur.execute("""
create temporary view a as 
    select * from prm_emi
    where Category = 'IPC1' and
          date <= '1990-12-31' and
          date >= '1900-01-01' and
          Entity = 'CO2' and
          Scenario = 'HISTCR' """)

In [14]:
cur.execute('drop view if exists b')
cur.execute("""create temporary view b as 
                  select * from prm_emi
                  where area in ('DEU', 'FIN', 'ZWE') """)

In [15]:
def pdq(query):
    return pd.read_sql_query(query, conn)

In [17]:
pdq("""select a.emissions + b.emissions as results
       from a join b using (area, category, date, entity, scenario)""")
# note the timing includes conversion into a pandas dataframe

,results
0,640000.0
1,638000.0
2,626000.0
3,674000.0
4,698000.0
...,...
268,22800.0
269,26000.0
270,27200.0
271,27400.0


In [18]:
pdq("""
select distinct primap_metadata_id, description, scenario, source
from primap_metadata join primap_data using (primap_metadata_id)
where primap_metadata.entity = 'N2O'
  and primap_data.area = 'FIN'
  and primap_metadata.category = 'IPC0'
order by primap_metadata_id
""")

,primap_metadata_id,Description,Scenario,Source
0,2993,Result of N2O_IPC5_TOTAL_NET_HISTCR_PRIMAPHIST...,HISTCR,PRIMAPHIST20
1,2997,Result of N2O_IPC5+IPC4+IPC3+IPC2_TOTAL_NET_HI...,HISTORY,CRF2019
2,2998,Result of N2O_IPC5+IPC4+IPC3+IPC2_TOTAL_NET_HI...,HISTORY,CRF2020
3,2999,Result of N2O_CAT7+CAT6+CAT5+CAT4+CAT3+CAT2_TO...,HISTORY,EDGAR42COMPI
4,3000,Result of N2O_CAT7+CAT6+CAT4+CAT3+CAT2_TOTAL_N...,HISTORY,EDGAR432I
5,3001,Result of N2O_IPC5+IPC4+IPC3+IPC2_TOTAL_NET_HI...,HISTORY,EDGAR50I
6,3004,Result of N2O_IPC5_TOTAL_NET_HISTTP_PRIMAPHIST...,HISTTP,PRIMAPHIST20
